In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json
import getpass
from datetime import datetime
import os
import pandas as pd
import re


In [2]:
class FacebookGroupMemberCrawl:
    def __init__(self, username, password, group_id, scroll_count):
        print("\n====== Facebook Group Member Scraper ======")
        self.email = username
        self.password = password
        self.group_id = group_id
        self.scroll_count = scroll_count
        self.setup_driver()

    def setup_driver(self):
        try:
            self.driver = webdriver.Chrome()
            self.driver.maximize_window()
        except Exception as e:
            print(f"Error: {e}")

    def login(self):
        try:
            self.driver.get("https://www.facebook.com/")
            self.driver.implicitly_wait(10)
            self.driver.find_element(By.ID, "email").send_keys(self.email)
            self.driver.find_element(By.ID, "pass").send_keys(self.password)
            self.driver.find_element(By.NAME, "login").click()
            time.sleep(10)
            print('Login success')
            return True
        except Exception as e:
            print(f"Error: {e}")
            return False

    def get_group_members(self):
        try:
            self.driver.get(f"https://www.facebook.com/groups/{self.group_id}/members")
            time.sleep(5)
            members = set()
            for i in range(self.scroll_count):
                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(3)
                print(f"Scroll {i+1}/{self.scroll_count}")
                user_elements = self.driver.find_elements(By.CSS_SELECTOR, "a[href*='/user/']")
                print(len(user_elements))
                for user in user_elements:
                    try:
                        href = user.get_attribute("href")
                        if '/user/' in href:
                            user_id = href.split('/user/')[1].split('?')[0].strip('/')
                            name = user.text
                            members.add((user_id, name))
                            print(f"Member: {user_id} - {name}")
                    except Exception as e:
                        continue
            return list(members)
        
        except Exception as e:
            print(f"Error: {e}")

    def save_to_excel(self, members):
        try:
            file_name = f"all_group_members.xlsx"
            df = pd.DataFrame(members, columns=["User ID", "Name"])
            df_clean = df[df['Name'].str.strip() != '']
            df_clean.to_excel(file_name, index=False)
            print(f"Data saved to {file_name}")
        except Exception as e:
            print(f"Error: {e}")


In [3]:
class FacebookGroupPostCrawl:
    def __init__(self, username, password, group_id, scroll_count):
        print("\n====== Facebook Group post Scraper ======")
        self.email = username
        self.password = password
        self.group_id = group_id
        self.scroll_count = scroll_count
        self.setup_driver()

    def setup_driver(self):
        try:
            self.driver = webdriver.Chrome()
            self.driver.maximize_window()
        except Exception as e:
            print(f"Error: {e}")

    def login(self):
        try:
            self.driver.get("https://www.facebook.com/")
            self.driver.implicitly_wait(10)
            self.driver.find_element(By.ID, "email").send_keys(self.email)
            self.driver.find_element(By.ID, "pass").send_keys(self.password)
            self.driver.find_element(By.NAME, "login").click()
            time.sleep(10)
            print('Login success')
            return True
        except Exception as e:
            print(f"Error: {e}")
            return False
    def get_group_posts(self):
        try:
            self.driver.get(f"https://www.facebook.com/groups/{self.group_id}/?sorting_setting=TOP_POSTS")
            time.sleep(5)
            postlist = set()
            last_height = self.driver.execute_script("return document.body.scrollHeight")

            for i in range(self.scroll_count):
                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(3) 
                post_elements = self.driver.find_elements(By.XPATH, "//div[contains(@role, 'article')]")
                print(f"Found {len(post_elements)} post elements on scroll {i+1}")
                for post in post_elements:
                    try:
                        content = post.text.strip()
                        post_user = content.split('\n')[0]
                        post_content = content.split('\n')[1]
                        post_id_element = post.find_element(By.XPATH, ".//a[contains(@href, '/posts/')]")
                        post_id = post_id_element.get_attribute("href").split("/")[-2]
                        postlist.add((post_id,post_user,post_content))
                    except Exception as e:
                        continue
            return list(postlist)
        except Exception as e:
            print(f"{e}")
    def save_to_excel(self,postlist):
        try:
            file_name = f"ex_post.xlsx"
            df = pd.DataFrame(postlist, columns=["post_id", "post_user",'post_content'])
            df.to_excel(file_name, index=False)
            print(f"Post data saved to {file_name}")
        except Exception as e:
            print(f"Error saving posts to Excel: {e}")
        

In [4]:
username = 'phanmthihoa05@gmail.com'
password = 'Hung26082003'
group_id = '811896080494851'
scroll_count = 5
try:
    scraper = FacebookGroupPostCrawl(username= username, password = password, group_id = group_id, scroll_count= scroll_count)
    if scraper.login():
        print('-----------------')
        postlist = scraper.get_group_posts()
        scraper.save_to_excel(postlist)
        time.sleep(10)
except Exception as e:
    pass


====== Facebook Group post Scraper ======
Login success
-----------------
Found 3 post elements on scroll 1
Found 7 post elements on scroll 2
Found 4 post elements on scroll 3
Found 8 post elements on scroll 4
Found 11 post elements on scroll 5
Post data saved to ex_post.xlsx


In [5]:
print(members)

NameError: name 'members' is not defined

In [5]:
# def save_to_excel(members):
#             file_name = f"ex_post.xlsx"
#             df = pd.DataFrame(members, columns=["Post ID", "type"])
#             df.to_excel(file_name, index=False)
#             print(f"Data saved to {file_name}")
# save_to_excel(all_post)

In [6]:
# import pandas as pd

# # Đọc file Excel
# df = pd.read_excel('ex_post.xlsx')

# # In từng dòng dưới dạng dictionary
# for index, row in df.iterrows():
#     print(row.to_dict())
